In [ ]:
import numpy as np
import scipy as sp
import pandas as pd
import datetime

from compute import *

In [ ]:
#constants
ProjectName = "Tower St 4plex and 2 adus"
# assumptions
NumberUnits=4
price     = 1300000
Carry     = 0
rent      = 2000*2+1400*2
ltv       = 80
loanRatePer = 3.5
expenseRate = 30
rentPF      = 2200*2+1500*2+2000*2
# Proforma
expenseRatePF=30
capPF        =4.5
repairCost   = 500000

computeProjectSimple(ProjectName, NumberUnits, price, rent,expenseRate, Carry,ltv, loanRatePer, rentPF,expenseRatePF,capPF,repairCost)



In [ ]:
dfAct = pd.DataFrame(columns=["Value","Monthly","Annual","%"])

dfAct.loc["Purchase Price","Value"] = price
dfAct.loc["closing cost","%"]       = 1
dfAct.loc["Repair","Value"]         = 5000
dfAct.loc["Reserve","Value"]        = 6000*5
dfAct.loc["Rent Roll","Monthly"] = rent
dfAct.loc["Loan Rate","%"]  = 3.5
dfAct.loc["LTV","%"]        = 75
dfAct.loc["Number Of Units","Value"]=4
simpleExpense = 1
dfAct.loc["Util Per Unit","Value"]  =0
dfAct.loc["Insurance","%"]  =0.2
dfAct.loc["Expenses","%"]   =30
dfAct.loc["Taxes","%"]      = 1.25
dfAct.loc["Vacancy","%"]    = 4
dfAct.loc["Management","%"] = 5
dfAct.loc["Maintenance","%"]= 0.3

In [ ]:
#derived from 3rd avenue

dfact = setMonthlyAndAnnual(dfAct)
if simpleExpense ==1:
    noi=computeNoi(rent=dfAct.loc["Rent Roll","Monthly"],expenseRate=dfAct.loc["Expenses","%"])
    dfAct.loc["NOI","Annual"] = noi
    dfAct.loc["Expenses","Monthly"]=dfAct.loc["Expenses","%"]/100. * dfAct.loc["Rent Roll","Monthly"]
else:
    dfAct.loc["Util","Monthly"] = dfAct.loc["Number Of Units","Value"] * dfAct.loc["Util Per Unit","Value"]
    for c in ["Vacancy","Management"]:
        dfAct.loc[c,"Monthly"] = dfAct.loc[c,"%"]/100.*dfAct.loc["Rent Roll","Monthly"]
        dfAct.loc[c,"Monthly"] = dfAct.loc[c,"%"]/100.*dfAct.loc["Rent Roll","Monthly"]
    for c in ["Maintenance","Taxes","Insurance"]:
        dfAct.loc[c,"Monthly"]= dfAct.loc[c,"%"]/100.*dfAct.loc["Purchase Price","Value"]/12.
    dfAct.loc["Expenses","Monthly"] = sum(dfAct.loc[["Vacancy","Management","Maintenance","Taxes","Insurance","Util"],"Monthly"])
    dfAct.loc["NOI","Monthly"]      = dfAct.loc["Rent Roll","Monthly"] - dfAct.loc["Expenses","Monthly"]
    dfAct.loc["Expense","%"] = dfAct.loc["Expenses","Monthly"]/ dfAct.loc["Rent Roll","Monthly"]
    
dfAct.loc["closing cost","Value"]=dfAct.loc["Purchase Price","Value"] * dfAct.loc["closing cost","%"]/100.
dfAct.loc["Cap","%"] = computeCap(
    rent    =dfAct.loc["Rent Roll","Annual"],
    price   =dfAct.loc["Repair","Value"]+dfAct.loc["Purchase Price","Value"] ,
    expenses=dfAct.loc["Expenses","Annual"])    

# loan payment
pmt =  -1*np.pmt(
    dfAct.loc["Loan Rate","%"]/100/12.,
    360,dfAct.loc["Purchase Price","Value"] *dfAct.loc["LTV","%"]/100)
dfAct.loc["Debt Service","Monthly"] = pmt
dfAct.loc["DSCR","%"] = dfAct.loc["NOI","Monthly"] / dfAct.loc["Debt Service","Monthly"] 
dfact = setMonthlyAndAnnual(dfAct)

dfAct.loc["Cash Flow","Annual"] = (dfAct.loc["NOI","Annual"] - dfAct.loc["Debt Service","Annual"]) 

dfAct.loc["Out Of Pocket","Value"]=\
    dfAct.loc["closing cost","Value"]+\
    dfAct.loc["Repair","Value"]+\
    dfAct.loc["Reserve","Value"]+\
    dfAct.loc["Purchase Price","Value"]*(100-dfAct.loc["LTV","%"])/100.
dfAct.loc["Cash Flow","%"] = dfAct.loc["Cash Flow","Annual"] / dfAct.loc["Out Of Pocket","Value"] * 100
dfact = setMonthlyAndAnnual(dfAct)
dfAct

In [ ]:
computeCap(
    rent    =dfAct.loc["Rent Roll","Annual"],
    price   =1.3e6,
    expenses=dfAct.loc["Expenses","Annual"])    


In [ ]:
simpleDepreciation=1
rentIncrease=5        # rent increase
maintenanceIncrease=3 # for all OI
years =datetime.datetime.today().year + np.arange(0,10)
dfPF = pd.DataFrame(columns=years)

updates = {'year':1,"expenseRate":30,"repairCost":450000, "rent":2200*2+1500*2+2000*2}

for year in years:
    if year == years[0]:
        dfPF.loc["Rent Roll",year]    = dfAct.loc["Rent Roll","Annual"]
        dfPF.loc["Expenses", year]    = -1*dfAct.loc["Expenses","Annual"]
    elif year == years[0] +updates["year"]:
        dfPF.loc["Rent Roll",year]    = updates["rent"]*12
        dfPF.loc["Expenses",year]     = -1*dfPF.loc["Rent Roll",year] * updates["expenseRate"]/100.
    else:
        dfPF.loc["Rent Roll",year]    = dfPF.loc["Rent Roll",year-1] *(100+rentIncrease)/100.
        dfPF.loc["Expenses",year]     = dfPF.loc["Expenses",year-1]  *(100+maintenanceIncrease)/100.
        
    dfPF.loc["NOI",year]          = dfPF.loc["Rent Roll",year]+dfPF.loc["Expenses",year]
    dfPF.loc["Debt Service",year] = computePmt(dfAct,year=1)*12
    if simpleDepreciation == 1:
        dfPF.loc["Depreciation",year] = (dfAct.loc["Repair","Value"]+dfAct.loc["Purchase Price","Value"])* -2./3./27.5    
    # Cash Flow NOI minus debt service
    dfPF.loc["Cash Flow",year]    = dfPF.loc["NOI",year] + dfPF.loc["Debt Service",year]
    # Profit=NOI minus interest payment
    dfPF.loc["Profit",year]       = dfPF.loc["NOI",year] + computePmt(dfAct,year=1,ptype=LoanPaymentType.Interest)*12    
    # Pre Tax Profit = profit - minus depreciation
    dfPF.loc["Pre-Tax Profit",year]   = dfPF.loc["Profit",year] + dfPF.loc["Depreciation",year]
    dfPF.loc["Cap Rate",year]         = dfPF.loc["NOI",year]      / dfAct.loc["Purchase Price","Value"]*100
    # cash on cash = profit / out of pocket acquisition
    dfPF.loc["Cash on Cash",year]     = dfPF.loc["Profit",year]/ dfAct.loc["Out Of Pocket","Value"] * 100
    #dfPF.loc["Cash Flow","%"] = dfPF.loc["Cash Flow","Annual"] / dfAct.loc["Out Of Pocket","Value"] * 100
dfPF